In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
import pickle

In [2]:
company= 'ICICIBANK.NS'
startDate= '2021-09-01'
endDate= '2023-08-20'

In [3]:
df= yf.download(company, interval='1h', start=startDate, end= endDate)
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-09-01 09:15:00+05:30,722.150024,734.750000,720.099976,733.599976,733.599976,0
2021-09-01 10:15:00+05:30,733.599976,734.400024,725.000000,726.599976,726.599976,2155281
2021-09-01 11:15:00+05:30,726.599976,727.450012,720.549988,721.299988,721.299988,1415091
2021-09-01 12:15:00+05:30,721.299988,724.500000,718.950012,723.700012,723.700012,1396670
2021-09-01 13:15:00+05:30,723.599976,725.849976,721.750000,724.500000,724.500000,1152705
...,...,...,...,...,...,...
2023-08-18 11:15:00+05:30,951.400024,951.799988,949.599976,951.349976,951.349976,1102002
2023-08-18 12:15:00+05:30,951.400024,953.200012,950.299988,952.500000,952.500000,1065889
2023-08-18 13:15:00+05:30,952.549988,954.299988,950.799988,950.900024,950.900024,2807318


In [4]:
trainSet= df.iloc[:, 1:2].values

In [5]:
df.shape

(3395, 6)

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler= MinMaxScaler(feature_range=(0,1))

In [7]:
trainingSetScaled= scaler.fit_transform(df['Close'].values.reshape(-1,1))

In [8]:
xTRAIN = []
yTRAIN = []

for i in range(100, len(trainingSetScaled)):
    xTRAIN.append(trainingSetScaled[i-60:i, 0])
    yTRAIN.append(trainingSetScaled[i, 0])
    
xTRAIN, yTRAIN= np.array(xTRAIN), np.array(yTRAIN)

In [9]:
xTRAIN= np.reshape(xTRAIN, newshape=(xTRAIN.shape[0], xTRAIN.shape[1], 1))

In [10]:
trainingSetScaled.shape

(3395, 1)

In [11]:
xTRAIN.shape

(3295, 60, 1)

In [12]:
yTRAIN.shape

(3295,)

In [13]:
import tensorflow as tf
from tensorflow import keras


if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")

In [14]:
regressor= keras.models.Sequential([
    keras.layers.LSTM(units=72, return_sequences= True, input_shape= (xTRAIN.shape[1], 1)),
    keras.layers.Dropout(rate= 0.2),
    keras.layers.LSTM(units=60, return_sequences= True), 
    keras.layers.Dropout(rate= 0.2),
    keras.layers.LSTM(units=50), 
    keras.layers.Dropout(rate= 0.2),
    keras.layers.Dense(units= 1)
])

In [15]:
regressor.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 72)            21312     
                                                                 
 dropout (Dropout)           (None, 60, 72)            0         
                                                                 
 lstm_1 (LSTM)               (None, 60, 60)            31920     
                                                                 
 dropout_1 (Dropout)         (None, 60, 60)            0         
                                                                 
 lstm_2 (LSTM)               (None, 50)                22200     
                                                                 
 dropout_2 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 5

In [16]:
regressor.compile(optimizer= 'adam', loss= keras.losses.mean_squared_error, metrics=['accuracy'])

In [ ]:
regressor.fit(x= xTRAIN, y=yTRAIN, batch_size= 32, epochs= 120)

Epoch 1/120
103/103 [==============================] - 8s 23ms/step - loss: 0.0141 - accuracy: 6.0698e-04
Epoch 2/120
103/103 [==============================] - 2s 21ms/step - loss: 0.0039 - accuracy: 6.0698e-04
Epoch 3/120
 72/103 [===================>..........] - ETA: 0s - loss: 0.0035 - accuracy: 8.6806e-04

In [ ]:
prdctn= regressor.predict(xTRAIN)

regressor.save(f"../model/exports/{company}/")

In [ ]:
testDF= yf.download(company, interval='1h', start=endDate, end=datetime.datetime.now())
testDF

In [ ]:
realCP= testDF['Close'].values

In [ ]:
dfTotal= pd.concat((df['Close'], testDF['Close']), axis=0)

In [ ]:
modelInp= dfTotal[len(dfTotal)-len(testDF)-60:].values

In [ ]:
modelInp= modelInp.reshape(-1,1)

In [ ]:
modelInp= scaler.transform(modelInp)

In [ ]:
xTEST= []

for i in range(60, len(modelInp)):
    xTEST.append(modelInp[i-60:i, 0])

xTEST= np.array(xTEST)

In [ ]:
xTEST.shape

In [ ]:
xTEST= np.reshape(xTEST, newshape= (xTEST.shape[0], xTEST.shape[1], 1))

In [ ]:
#pred = pred[..., 0]

In [ ]:
pred= regressor.predict(xTEST)

In [ ]:
pred= scaler.inverse_transform(pred)

In [ ]:
pred.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(realCP, color= 'r', label= 'real close price')
plt.plot(pred, color= 'b', label= 'predicted close price')
plt.title(company)
plt.legend()
plt.show()

In [ ]:
realData= [modelInp[len(modelInp)+1-60:len(modelInp+1), 0]]

In [ ]:
realData= np.array(realData)

In [ ]:
realData= np.reshape(realData, newshape=(realData.shape[0], realData.shape[1], 1))

In [ ]:
prediction= regressor.predict(realData)

In [ ]:
prediction= scaler.inverse_transform(prediction)

In [ ]:
prediction

print("attempting to save scaler values.........")
scaler_path= f"D:\Code\Projects\stockSenseAPIv2\hourlyExports\scalers\{company}-scaler.pkl"
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)

print("attempting to save dataframe.........")
data_path= f"D:\Code\Projects\stockSenseAPIv2\hourlyExports\datas\{company}-data.csv"
df.to_csv(data_path)

print("attempting to save model...........")
# Load your Keras model
#model = tf.keras.models.load_model('../model/exports/ITC.NS/')

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(regressor)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.experimental_lower_tensor_list_ops = False  # Disable lowering of tensor list ops
tflite_model = converter.convert()

# Save the converted TensorFlow Lite model
tflite_model_path = f"D:\Code\Projects\stockSenseAPIv2\hourlyExports\{company}.tflite"
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)
print("assets saving completed!")